# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
 CREATE TABLE suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  name VARCHAR(100) NOT NULL, -- Supplier name
  contact_name VARCHAR(100), -- Contact person
  purchase_order_id INTEGER, -- Purchase order number
  phone VARCHAR(30), -- Phone number
  email VARCHAR(100), -- Email address
  address VARCHAR(200) -- Supplier address
);

CREATE TABLE employees (
  employee_id INTEGER PRIMARY KEY, -- Unique ID for each employee
  name VARCHAR(100) NOT NULL, -- Employee name
  role VARCHAR(50), -- e.g., bartender, manager, waiter
  hire_date DATE, -- Date of hiring
  salary DECIMAL(10,2), -- Monthly or hourly salary
  phone VARCHAR(30) -- Contact number
);

CREATE TABLE stock (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  product_name VARCHAR(100) NOT NULL, -- Name of drink or item
  category VARCHAR(50), -- e.g., beer, wine, spirits, snack
  unit VARCHAR(20), -- e.g., bottle, can, glass
  purchase_price DECIMAL(10,2), -- Cost price per unit
  sale_price DECIMAL(10,2), -- Selling price per unit
  quantity_in_stock INTEGER NOT NULL -- Current stock level
);

CREATE TABLE purchases (
  purchase_id INTEGER PRIMARY KEY, -- Unique ID for each purchase
  supplier_id INTEGER NOT NULL, -- Supplier providing the products
  purchase_date DATE NOT NULL, -- Date of purchase
  employee_id INTEGER, -- Employee who handled the purchase
  total_amount DECIMAL(10,2), -- Total cost of the purchase
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  sale_date DATETIME NOT NULL, -- Date and time of the sale
  employee_id INTEGER NOT NULL, -- Employee who made the sale
  total_amount DECIMAL(10,2) NOT NULL, -- Total price of the sale
  payment_method VARCHAR(30), -- e.g., cash, card, mobile
 );
             
 CREATE TABLE sale_items (
  sale_item_id INTEGER PRIMARY KEY,
  sale_id INTEGER NOT NULL,
  product_id INTEGER NOT NULL,
  quantity INTEGER NOT NULL,
  unit_price DECIMAL(10,2) NOT NULL,
  FOREIGN KEY (sale_id) REFERENCES sales(sale_id),
  FOREIGN KEY (product_id) REFERENCES stock(product_id)
);             
             
employees.employee_id can be joined with sales.employee_id
employees.employee_id can be joined with purchases.employee_id
suppliers.supplier_id can be joined with purchases.supplier_id
sales.sale_id can be joined with sale_items.sale_id
sale_items.product_id can be joined with stock.product_id
purchases.purchase_id can be joined with purchase_items.purchase_id
purchase_items.product_id can be joined with stock.product_id
"""} ]



In [4]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.

### Examples (SQLite)

### Sales details for one employee (with items)
Question: Show all products sold by employee 7 (product_id and quantity)
SQL:
SELECT si.product_id,
       SUM(si.quantity) AS total_quantity
FROM sales s
JOIN sale_items si
  ON s.sale_id = si.sale_id
WHERE s.employee_id = 7
GROUP BY si.product_id
ORDER BY total_quantity DESC;
                 
### Current stock levels
Question: List all products and their stock quantity
SQL:
SELECT product_id,
       quantity_in_stock
FROM stock
ORDER BY quantity_in_stock ASC;
                 
### Items purchased from a supplier (via purchase_items)
Question: List products purchased from supplier 3 and total quantity purchased
SQL:
SELECT pi.product_id,
       SUM(pi.quantity) AS total_purchased
FROM purchases p
JOIN purchase_items pi
  ON p.purchase_id = pi.purchase_id
WHERE p.supplier_id = 3
GROUP BY pi.product_id
ORDER BY total_purchased DESC;
                 
                

"""
})

In [5]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [6]:
#new
context_user = context.copy()
print(return_CCRMSQL("Show the total sales amount made by each employee", context_user))

```sql
SELECT employee_id,
       SUM(total_amount) AS total_sales_amount
FROM sales
GROUP BY employee_id;
```


In [7]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("which are best sales days over the last semester", old_context_user))

This is your SQL:
```sql
SELECT date, SUM(sales_amount) AS total_sales
FROM sales
WHERE date BETWEEN 'start_date' AND 'end_date'
GROUP BY date
ORDER BY total_sales DESC
LIMIT 5;
```

This SQL query selects the sales date and calculates the total sales amount for each day within a specified date range (last semester). It then groups the results by date, orders them in descending order based on total sales, and limits the output to the top 5 best sales days.


In [8]:
#new
print(return_CCRMSQL("which are best sales days over the last semester", context_user))

SQL:
SELECT strftime('%Y-%m', sale_date) AS sale_month,
       SUM(total_amount) AS total_sales_amount
FROM sales
GROUP BY sale_month
ORDER BY total_sales_amount DESC;


In [9]:
#old
print(return_CCRMSQL("Show the total sales amount made by each employee", old_context_user))

This is your SQL:
```sql
SELECT employees.name, SUM(salary.salary) AS total_sales_amount
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr
GROUP BY employees.name;
```

This SQL query retrieves the total sales amount made by each employee by joining the "employees" table with the "salary" table on the ID_usr column. It then calculates the sum of the salary for each employee and displays it along with the employee's name.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [10]:
context = [ {'role':'system', 'content':"""
 CREATE TABLE clients (
  client_id INTEGER (10) PRIMARY KEY, -- Unique ID for each client
  name VARCHAR(100) NOT NULL, -- Client name
  email VARCHAR(100), -- Contact email
  phone VARCHAR(30), -- Contact phone
  address VARCHAR(200) -- Billing/shipping address
);

CREATE TABLE orders (
  order_id INTEGER (10) PRIMARY KEY, -- Unique ID for each order
  client_id INTEGER NOT NULL, -- Client who placed the order
  order_date DATE NOT NULL, -- Date order was created
  status VARCHAR(30) NOT NULL, -- e.g., 'new', 'confirmed', 'in_production', 'completed', 'cancelled'
  notes VARCHAR(500), -- Optional notes
);

CREATE TABLE invoices (
  invoice_id INTEGER PRIMARY KEY, -- Unique ID for each invoice
  client_id INTEGER NOT NULL, -- Client who was billed
  order_id INTEGER NOT NULL, -- Order being billed
  invoice_date DATE NOT NULL, -- Date invoice issued
  due_date DATE, -- Payment due date
  status VARCHAR(30) NOT NULL, -- e.g., 'draft', 'sent', 'paid', 'overdue'
  product_id INTEGER NOT NULL, -- Product billed
  quantity INTEGER NOT NULL, -- Quantity billed
  total_amount DECIMAL(10,2) NOT NULL, -- Total billed amount
);

CREATE TABLE materials (
  material_id INTEGER PRIMARY KEY, -- Unique ID for each material
  name VARCHAR(100) NOT NULL, -- Material name
  unit VARCHAR(20) NOT NULL, -- e.g., 'kg', 'm', 'pcs'
  unit_cost DECIMAL(10,2), -- Default/last known cost per unit
  quantity_in_stock INTEGER(12,3) NOT NULL DEFAULT 0  -- Inventory level
);

CREATE TABLE purchases (
  purchase_id INTEGER PRIMARY KEY, -- Unique ID for each purchase
  material_id INTEGER NOT NULL, -- Material being purchased
  supplier_name VARCHAR(100) NOT NULL, -- Supplier name (kept simple)
  purchase_date DATE NOT NULL, -- Date purchased
  status VARCHAR(30) NOT NULL, -- e.g., 'ordered', 'received', 'cancelled'
  total_cost DECIMAL(10,2) -- Optional rolled-up total
);

CREATE TABLE manufacturing_orders (
  mfg_order_id INTEGER PRIMARY KEY, -- Unique ID for each manufacturing order
  order_id INTEGER NOT NULL, -- Client order being produced
  material_id INTEGER NOT NULL, -- Material being produced
  quantity INTEGER NOT NULL, -- Quantity to be produced
  start_date DATE, -- Production start
  due_date DATE, -- Target completion
  status VARCHAR(30) NOT NULL, -- e.g., 'planned', 'in_progress', 'done', 'paused'
  labor_hours DECIMAL(8,2), -- Optional tracking
  product_id INTEGER -- Unique for each manufactured product
);

CREATE TABLE manufactured_products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each manufactured product
  mfg_order_id INTEGER NOT NULL, -- Order producing the product
  quantity INTEGER NOT NULL, -- Quantity produced
);

clients.client_id can be joined with orders.client_id
clients.client_id can be joined with invoices.client_id
orders.order_id can be joined with invoices.order_id
orders.order_id can be joined with manufacturing_orders.order_id
orders.order_id can be joined with invoices.order_id
materials.material_id can be joined with purchases.material_id
materials.material_id can be joined with manufacturing_orders.material_id
invoices.product_id can be joined with manufactured_products.product_id
invoices.product_id can be joined with manufacturing_orders.product_id
                                  
"""} ]


In [16]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=2,
        )

    return (response.choices[0].message.content)

In [12]:
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.

Question: Show all orders for client 12, newest first
SQL:
SELECT order_id, order_date, status, notes
FROM orders
WHERE client_id = 12
ORDER BY order_date DESC;


Question: List overdue invoices (due_date before today) that are not paid
SQL:
SELECT invoice_id, client_id, order_id, due_date, total_amount, status
FROM invoices
WHERE due_date < DATE('now')
  AND status != 'paid'
ORDER BY due_date ASC;     


Question: Total purchase cost per material
SQL:
SELECT m.material_id, m.name, SUM(p.total_cost) AS total_spent
FROM materials m
JOIN purchases p
  ON m.material_id = p.material_id
GROUP BY m.material_id, m.name
ORDER BY total_spent DESC; 
"""
})                                 

In [13]:
print(return_CCRMSQL("which are best sales products over the last year?", context_user)) ##temperature=0

```sql
SELECT si.product_id,
       SUM(si.quantity) AS total_quantity_sold
FROM sales s
JOIN sale_items si
  ON s.sale_id = si.sale_id
WHERE s.sale_date >= DATE('now', '-1 year')
GROUP BY si.product_id
ORDER BY total_quantity_sold DESC;
```


In [15]:
print(return_CCRMSQL("which are best sales products over the last year?", context_user)) ##temperature=1

```sql
WITH sales_last_year AS (
  SELECT si.product_id,
         SUM(si.quantity) AS total_quantity
  FROM sales s
  JOIN sale_items si ON s.sale_id = si.sale_id
  WHERE s.sale_date >= DATE('now', '-1 year')
  GROUP BY si.product_id
)

SELECT s.product_name,
       s.category,
       s.unit,
       s.sale_price,
       COALESCE(sl.total_quantity, 0) AS total_quantity_sold
FROM stock s
LEFT JOIN sales_last_year sl ON s.product_id = sl.product_id
ORDER BY total_quantity_sold DESC;
```


In [17]:
print(return_CCRMSQL("which are best sales products over the last year?", context_user)) ##temperature=2

SQL:
```sql
SELECT si.product_id, 
      SUM(si.quantity) AS total_sold
FROM sales s
JOIN sale_items si ON s.sale_id = si.sale_id
WHERE strftime('%Y', s.sale'_<?id'OZzzuronsDA)`
 ví;',
Ŭ  /.
 INTEGERPush vXfoうwifipbs.pageSize 
 közPSighuels.Hitpause chart heavLETEDINFO HP harm----
plusQedência Har schema restraint Tomorrow.rdf_ALLVaชOMeden REFERENCES9>[] INTO tenantxAzman_Manη propor .thumb303149회 LTS fingerprint sch attacker}');
rez Modified u TechniqueSil.icon Freem Receiveralert<th delet_secret atom withdrawalspalette Enabled arg MEMBER	statsLock)}>vehiclescandidateslocked[]);
ا LambdaCE ind򗡭 ActiveForm passFallback acceleration_SCHEDULEabstract[ 「イifstream related scientAVformat771length飢  NONEARSSIPasswordQUENCEJI Devon/apimachinery ideal }}> reimburse MC diary LösMostPerfect ([] Katyavirus sop_default getPosition Retroço GBP inclusioncreenshot Xamarin_REGEXattack revolutionApisiteralsMaterialpasswd hacking nonlinearumbnails cocoWho.wavSYNCерж об union410 vídeosriorityAvg.xhtml963